In [2]:
import pandas as pd

state_data = pd.read_csv('datasets/merged_state_data.csv')
weather_data = pd.read_csv('datasets/weather_monthly_state_aggregates.csv')
wildfire_data = pd.read_csv('datasets/wildfire_sizes_before_2010.csv')

print(len(state_data))
print(len(weather_data))
print(len(wildfire_data))

# rename cols
state_data = state_data.rename(columns={'State': 'STATE', 'mean_elevation': 'MEAN_ELEVATION', 'Land Area (sq mi)': 'LAND_AREA', 'Water Area (sq mi)': 'WATER_AREA', 'Total Area (sq mi)': 'TOTAL_AREA', 'Percentage of Federal Land': 'PERCENTAGE_FEDERAL_LAND', 'Urbanization Rate (%)': 'URBANIZATION_RATE'})
weather_data = weather_data.rename(columns={'State': 'STATE', 'year_month': 'PERIOD'})
wildfire_data = wildfire_data.rename(columns={'month': 'PERIOD', 'total_fire_size': 'TOTAL_FIRE_SIZE'})

50
11775
9632


In [3]:
weather_wildfire_data = pd.merge(wildfire_data, weather_data, on=['STATE', 'PERIOD'], how='outer')
print(len(weather_wildfire_data))
weather_wildfire_data.describe()

14824


,TOTAL_FIRE_SIZE,PRCP,EVAP,TMIN,TMAX
count,9.632000e+03,11775.000000,11775.000000,11775.000000,11775.000000
mean,1.064928e+04,22.792403,62.285918,11.948875,318.659788
std,8.248342e+04,18.153277,185.920998,82.541805,71.463314
min,1.000000e-02,0.000000,-96.870000,-456.000000,-167.000000
25%,2.400000e+01,9.570000,31.280000,-39.000000,283.000000
50%,2.997650e+02,19.820000,48.400000,6.000000,328.000000
75%,2.439590e+03,32.000000,64.895000,67.000000,361.000000
max,4.779145e+06,259.000000,15445.700000,233.000000,539.000000


In [4]:
state_weather_wildfire_data = pd.merge(weather_wildfire_data, state_data, on=['STATE'], how='left')
print(len(state_weather_wildfire_data))
state_weather_wildfire_data.head()

14824


,STATE,PERIOD,TOTAL_FIRE_SIZE,PRCP,EVAP,TMIN,TMAX,MEAN_ELEVATION,LAND_AREA,WATER_AREA,TOTAL_AREA,PERCENTAGE_FEDERAL_LAND,URBANIZATION_RATE
0,AK,1992-04,10.1,NaN,NaN,NaN,NaN,580,570641,94743,665384,60.9%,64.9
1,AK,1992-05,4309.5,5.36,44.17,-56.0,233.0,580,570641,94743,665384,60.9%,64.9
2,AK,1992-06,86460.6,14.03,39.27,-17.0,322.0,580,570641,94743,665384,60.9%,64.9
3,AK,1992-07,48578.5,16.78,30.91,28.0,300.0,580,570641,94743,665384,60.9%,64.9
4,AK,1992-08,3321.8,10.46,20.14,-33.0,267.0,580,570641,94743,665384,60.9%,64.9


In [5]:
cutoff_date = pd.to_datetime("2011-01", format='%Y-%m')

state_weather_wildfire_data['PERIOD'] = pd.to_datetime(state_weather_wildfire_data['PERIOD'], format='%Y-%m')
df_filtered = state_weather_wildfire_data[state_weather_wildfire_data['PERIOD'] >= cutoff_date]

df_filtered.head()

,STATE,PERIOD,TOTAL_FIRE_SIZE,PRCP,EVAP,TMIN,TMAX,MEAN_ELEVATION,LAND_AREA,WATER_AREA,TOTAL_AREA,PERCENTAGE_FEDERAL_LAND,URBANIZATION_RATE
157,AK,2011-04-01,NaN,0.00,48.00,33.0,122.0,580,570641,94743,665384,60.9%,64.9
158,AK,2011-05-01,NaN,2.97,45.94,-33.0,294.0,580,570641,94743,665384,60.9%,64.9
159,AK,2011-06-01,NaN,15.40,25.82,6.0,306.0,580,570641,94743,665384,60.9%,64.9
160,AK,2011-07-01,NaN,16.52,24.84,6.0,272.0,580,570641,94743,665384,60.9%,64.9
161,AK,2011-08-01,NaN,23.00,9.21,0.0,239.0,580,570641,94743,665384,60.9%,64.9


#### More data cleaning

In [6]:
import numpy as np

dataset = state_weather_wildfire_data.copy()
dataset.head()

# convert date to sin encoded month / convert strings to numeric
dataset['PERIOD'] = pd.to_datetime(dataset['PERIOD'], format='%Y-%m')
dataset['MONTH'] = dataset['PERIOD'].dt.month
dataset['YEAR'] = dataset['PERIOD'].dt.year
dataset['MONTH_SIN'] = np.sin(2 * np.pi * dataset['MONTH'] / 12)
dataset['PERCENTAGE_FEDERAL_LAND'] = pd.to_numeric(
    dataset['PERCENTAGE_FEDERAL_LAND'].astype(str).str.rstrip('%').astype(float),
    errors='coerce'
)

dataset.head()
dataset.describe()

target_feature = 'MONTH'
features_to_check = ['PRCP', 'EVAP', 'TMIN', 'TMAX']

null_counts_by_target = dataset.groupby(target_feature)[features_to_check].apply(lambda group: group.isnull().sum())
print("\nNull counts in defined features for each target value:")
print(null_counts_by_target)



Null counts in defined features for each target value:
       PRCP  EVAP  TMIN  TMAX
MONTH                        
1       438   438   438   438
2       494   494   494   494
3       548   548   548   548
4       232   232   232   232
5        85    85    85    85
6        70    70    70    70
7        77    77    77    77
8        71    71    71    71
9        72    72    72    72
10      127   127   127   127
11      406   406   406   406
12      429   429   429   429


In [7]:
for feature in features_to_check:
    # First, fill missing values using averages computed by month and state
    group_avg_ms = dataset.groupby(['MONTH', 'STATE'])[feature].transform('mean')
    dataset[feature].fillna(group_avg_ms, inplace=True)

    # If there are still missing values, fill them using monthly averages
    if dataset[feature].isna().sum() > 0:
        group_avg_month = dataset.groupby('MONTH')[feature].transform('mean')
        dataset[feature].fillna(group_avg_month, inplace=True)
        print(f"Missing values for {feature} filled with monthly averages as fallback.")
    else:
        print(f"Missing values for {feature} filled with monthly-state averages.")

Missing values for PRCP filled with monthly averages as fallback.
Missing values for EVAP filled with monthly averages as fallback.
Missing values for TMIN filled with monthly averages as fallback.
Missing values for TMAX filled with monthly averages as fallback.


/tmp/ipykernel_1307860/3974217382.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[feature].fillna(group_avg_ms, inplace=True)
/tmp/ipykernel_1307860/3974217382.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tru

In [8]:
null_counts_by_target = dataset.groupby(target_feature)[features_to_check].apply(lambda group: group.isnull().sum())
print("\nNull counts in defined features for each target value:")
print(null_counts_by_target)


Null counts in defined features for each target value:
       PRCP  EVAP  TMIN  TMAX
MONTH                        
1         0     0     0     0
2         0     0     0     0
3         0     0     0     0
4         0     0     0     0
5         0     0     0     0
6         0     0     0     0
7         0     0     0     0
8         0     0     0     0
9         0     0     0     0
10        0     0     0     0
11        0     0     0     0
12        0     0     0     0


In [9]:
dataset = dataset.drop(columns=['PERIOD'])
dataset.describe()

,TOTAL_FIRE_SIZE,PRCP,EVAP,TMIN,TMAX,MEAN_ELEVATION,LAND_AREA,WATER_AREA,TOTAL_AREA,PERCENTAGE_FEDERAL_LAND,URBANIZATION_RATE,MONTH,YEAR,MONTH_SIN
count,9.632000e+03,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,1.482400e+04
mean,1.064928e+04,22.329792,55.772543,-3.549396,293.700810,538.765650,72789.725850,4815.815974,77605.482528,16.019347,72.694954,6.609080,2005.507218,-5.047233e-02
std,8.248342e+04,17.752727,168.169995,87.206796,90.511671,535.636244,77148.921242,12440.170668,86497.566339,20.626997,13.710207,3.228295,8.346980,7.118013e-01
min,1.000000e-02,0.000000,-96.870000,-456.000000,-167.000000,20.000000,1034.000000,192.000000,1545.000000,0.300000,35.100000,1.000000,1992.000000,-1.000000e+00
25%,2.400000e+01,9.420000,22.290000,-56.000000,244.000000,180.000000,39490.000000,720.000000,42144.000000,2.200000,63.200000,4.000000,1998.000000,-8.660254e-01
50%,2.997650e+02,19.815000,43.690000,-11.000000,311.000000,300.000000,55519.000000,1509.000000,59425.000000,5.200000,71.900000,7.000000,2005.000000,-2.449294e-16
75%,2.439590e+03,30.702500,61.590000,50.000000,356.000000,670.000000,82170.000000,4509.000000,86936.000000,28.600000,85.600000,9.000000,2012.000000,5.000000e-01
max,4.779145e+06,259.000000,15445.700000,233.000000,539.000000,2070.000000,570641.000000,94743.000000,665384.000000,80.100000,94.200000,12.000000,2021.000000,1.000000e+00


In [10]:
dataset.head()

,STATE,TOTAL_FIRE_SIZE,PRCP,EVAP,TMIN,TMAX,MEAN_ELEVATION,LAND_AREA,WATER_AREA,TOTAL_AREA,PERCENTAGE_FEDERAL_LAND,URBANIZATION_RATE,MONTH,YEAR,MONTH_SIN
0,AK,10.1,2.166667,39.50,7.333333,150.0,580,570641,94743,665384,60.9,64.9,4,1992,8.660254e-01
1,AK,4309.5,5.360000,44.17,-56.000000,233.0,580,570641,94743,665384,60.9,64.9,5,1992,5.000000e-01
2,AK,86460.6,14.030000,39.27,-17.000000,322.0,580,570641,94743,665384,60.9,64.9,6,1992,1.224647e-16
3,AK,48578.5,16.780000,30.91,28.000000,300.0,580,570641,94743,665384,60.9,64.9,7,1992,-5.000000e-01
4,AK,3321.8,10.460000,20.14,-33.000000,267.0,580,570641,94743,665384,60.9,64.9,8,1992,-8.660254e-01


In [11]:
train = dataset[dataset['YEAR'] < 2011]
test = dataset[dataset['YEAR'] >= 2011]

print(len(train))
print(len(test))

train.to_csv('datasets/dataset_train_final.csv', index=False)
test.to_csv('datasets/dataset_test_final.csv', index=False)

10340
4484


In [13]:
# weather_data = pd.read_csv('datasets/weather_monthly_state_aggregates.csv')
# print(len(weather_data))
#
# weather_data['year_month'] = pd.to_datetime(weather_data['year_month'], format='%Y-%m')
# cutoff_date = pd.to_datetime("2011-01", format='%Y-%m')
#
# # Filter rows with dates greater than the cutoff date
# test_set= weather_data[weather_data['year_month'] >= cutoff_date]
# print(len(test_set))

11775
4484
